In [1]:
from flashrag.config import Config
import os

reflection_path = '/shared/data3/yichi9/hotpot_dev_reflections.jsonl'

config = Config(config_file_path="./hotpotqa.yaml")
os.environ['CUDA_VISIBLE_DEVICES'] = "6,"

from flashrag.utils import get_dataset
dataset = get_dataset(config)

/shared/data3/yichi9/anaconda3/envs/flash/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import json
LOG_FILE = 'sayrag_early_results.jsonl'
def write_dict_to_jsonl(data_dict, file_path = LOG_FILE):
    """
    将一个字典写入到指定的JSON Lines文件中。

    :param file_path: JSONL文件的路径
    :param data_dict: 要写入的字典
    """
    if not isinstance(data_dict, dict):
        raise ValueError("data_dict必须是一个字典")

    with open(file_path, 'a', encoding='utf-8') as f:
        json_line = json.dumps(data_dict, ensure_ascii=False)
        f.write(json_line + '\n')

In [3]:
from sayrag import SayRAGPipeline
dataset = get_dataset(config)
pipeline = SayRAGPipeline(config)

2024-11-16 10:54:31,120	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


INFO 11-16 10:54:32 llm_engine.py:226] Initializing an LLM engine (v0.6.1.dev238+ge2c6e0a82) with config: model='mistralai/Mistral-7B-Instruct-v0.2', speculative_config=None, tokenizer='mistralai/Mistral-7B-Instruct-v0.2', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=mistralai/Mistral-7B-Instruct-v0.2, use_v2_block_manager=False, num_scheduler_steps=1

/shared/data3/yichi9/anaconda3/envs/flash/lib/python3.9/site-packages/vllm/transformers_utils/tokenizer_group/tokenizer_group.py:23: FutureWarning: It is strongly recommended to run mistral models with `--tokenizer_mode "mistral"` to ensure correct encoding and decoding.
  self.tokenizer = get_tokenizer(self.tokenizer_id, **tokenizer_config)


INFO 11-16 10:54:33 model_runner.py:1014] Starting to load model mistralai/Mistral-7B-Instruct-v0.2...
INFO 11-16 10:54:33 weight_utils.py:242] Using model weights format ['*.safetensors']


Loading safetensors checkpoint shards:   0% Completed | 0/3 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  33% Completed | 1/3 [00:00<00:01,  1.26it/s]
Loading safetensors checkpoint shards:  67% Completed | 2/3 [00:01<00:00,  1.06it/s]
Loading safetensors checkpoint shards: 100% Completed | 3/3 [00:02<00:00,  1.02it/s]
Loading safetensors checkpoint shards: 100% Completed | 3/3 [00:02<00:00,  1.05it/s]



INFO 11-16 10:54:37 model_runner.py:1025] Loading model weights took 13.4966 GB
INFO 11-16 10:54:42 gpu_executor.py:122] # GPU blocks: 11682, # CPU blocks: 2048
INFO 11-16 10:54:44 model_runner.py:1329] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 11-16 10:54:44 model_runner.py:1333] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 11-16 10:54:57 model_runner.py:1456] Graph capturing finished in 13 secs.


In [4]:
exp = 'baseline'
print(f"Running {exp}...")
filled_dataset = pipeline.run(
    dataset=dataset['dev'],
    reflection_path=reflection_path,
    augument_ratio=0.0,
    min_confidence_run_naive=114514, 
    min_confidence_drop_reflection=100, 
)

eval_result = pipeline.evaluator.evaluate(filled_dataset)
eval_result = {
    'exp': exp,
    **eval_result,
}

print(eval_result)
write_dict_to_jsonl(eval_result)

Running baseline...


Processed prompts: 100%|██████████| 500/500 [01:05<00:00,  7.62it/s, est. speed input: 7223.03 toks/s, output: 156.58 toks/s] 


{'em': 0.114, 'f1': 0.2336481221322797, 'acc': 0.346, 'precision': 0.21595853271435056, 'recall': 0.38584285714285715, 'avg_input_tokens': 827.588}
{'exp': 'baseline', 'em': 0.114, 'f1': 0.2336481221322797, 'acc': 0.346, 'precision': 0.21595853271435056, 'recall': 0.38584285714285715, 'avg_input_tokens': 827.588}


In [12]:
exp = 'baseline'
print(f"Running {exp}...")
filled_dataset = pipeline.run(
    dataset=dataset['dev'],
    reflection_path=reflection_path,
    augument_ratio=0.8,
    min_confidence_run_naive=114514, 
    min_confidence_drop_reflection=10, 
)

eval_result = pipeline.evaluator.evaluate(filled_dataset)
eval_result = {
    'exp': exp,
    **eval_result,
}

print(eval_result)
write_dict_to_jsonl(eval_result)

Running baseline...


Processed prompts: 100%|██████████| 500/500 [01:04<00:00,  7.75it/s, est. speed input: 7302.25 toks/s, output: 156.69 toks/s] 


{'em': 0.128, 'f1': 0.24722293031308779, 'acc': 0.378, 'precision': 0.23108117968651368, 'recall': 0.4105888888888888, 'avg_input_tokens': 822.522}
{'exp': 'baseline', 'em': 0.128, 'f1': 0.24722293031308779, 'acc': 0.378, 'precision': 0.23108117968651368, 'recall': 0.4105888888888888, 'avg_input_tokens': 822.522}
